In [61]:
img_path=r"D:\Lotus Labs Work\Anamoly Detection POC\Pallet Alignment\Images\0.jpg"

In [62]:
import cv2
import numpy as np

# Let's load a simple image with 3 black squares
image = cv2.imread(img_path)
#cv2.imshow('Input Image', image)
#cv2.waitKey(0)

# Grayscale
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)


# Find Canny edges
edged = cv2.Canny(gray, 30, 200)
#cv2.imshow('Canny Edges', edged)
#cv2.waitKey(0)

# Finding Contours
# Use a copy of your image e.g. edged.copy(), since findContours alters the image
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#cv2.imshow('Canny Edges After Contouring', edged)
#cv2.waitKey(0)

print("Number of Contours found = " + str(len(contours)))

# Draw all contours
# Use '-1' as the 3rd parameter to draw all
#cv2.drawContours(image, contours, -1, (0,255,0), thickness = 2)

#cv2.imshow('Contours', image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

Number of Contours found = 39


In [63]:
font = cv2.FONT_HERSHEY_COMPLEX
global_approx = []
control_group = []
for cnt in contours :

	approx = cv2.approxPolyDP(cnt, 0.009 * cv2.arcLength(cnt, True), True)
	
	# draws boundary of contours.
	#cv2.drawContours(image, [approx], 0, (0, 0, 255), 5)
	ga=[]
	for point in approx:
		#cv2.circle(image, point[0], 3,(0,0,255))
		control_group.append(point[0])
		print("POINT: ", point[0])
		for i, coord in enumerate(point[0]):
			if i%2==0:
				print('* new point x ', point[0][0].item())
				ga.append(point[0][0].item())
			else:
				print('* new point y ', point[0][1].item())
				ga.append(point[0][1].item())
				global_approx.append(ga)
				ga=[]
		

		
	# Used to flatted the array containing
	# the co-ordinates of the vertices.
	n = approx.ravel()
	i = 0

	for j in n :
		if(i % 2 == 0):
			x = n[i]
			y = n[i + 1]

			# String containing the co-ordinates.
			string = str(x) + " " + str(y)
			"""
			if(i == 0):
				# text on topmost co-ordinate.
				cv2.putText(image, "Arrow tip", (x, y),								font, 0.5, (255, 0, 0))
			else:
				# text on remaining co-ordinates.
				cv2.putText(image, string, (x, y),						font, 0.5, (0, 255, 0))
			"""
		i = i + 1

# Showing the final image.
#cv2.imshow('image2', image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

POINT:  [306 531]
* new point x  306
* new point y  531
POINT:  [310 679]
* new point x  310
* new point y  679
POINT:  [302 687]
* new point x  302
* new point y  687
POINT:  [312 680]
* new point x  312
* new point y  680
POINT:  [191 529]
* new point x  191
* new point y  529
POINT:  [192 529]
* new point x  192
* new point y  529
POINT:  [219 525]
* new point x  219
* new point y  525
POINT:  [220 525]
* new point x  220
* new point y  525
POINT:  [216 525]
* new point x  216
* new point y  525
POINT:  [215 526]
* new point x  215
* new point y  526
POINT:  [214 525]
* new point x  214
* new point y  525
POINT:  [213 526]
* new point x  213
* new point y  526
POINT:  [210 526]
* new point x  210
* new point y  526
POINT:  [209 527]
* new point x  209
* new point y  527
POINT:  [204 527]
* new point x  204
* new point y  527
POINT:  [203 528]
* new point x  203
* new point y  528
POINT:  [197 528]
* new point x  197
* new point y  528
POINT:  [196 529]
* new point x  196
* new point

In [64]:
print(len(global_approx))
print(len(control_group))
#global_approx

172
172


In [65]:
def find_sides(side, list_of_points):
    #list_of_points = np.array(list_of_points, dtype=np.int8)

    if side == 'up':

        return min(list_of_points, key=lambda y: (y[1], -y[1]))

    elif side == 'down':

        return max(list_of_points, key=lambda y: (y[1], -y[1]))

    elif side == 'left':

        return min(list_of_points, key=lambda x: (x[0], -x[1]))

    elif side == 'right':

        return max(list_of_points, key=lambda x: (x[0], -x[1]))

    else:
        print('error')

In [66]:
min_height = find_sides('up', global_approx)
max_height =  find_sides('down', global_approx)

print(max_height)
print(min_height)

[306, 690]
[268, 18]


In [67]:
total_height = max_height[1]-min_height[1]
total_height

672

In [68]:
max_height[1] - 0.1*total_height

622.8

In [69]:
bottom_left_corner = 0
bottom_right_corner = 0

In [70]:
# bottom left corner
temp_list = []
for point in global_approx:
    if point[1] > max_height[1] - 0.1*total_height:
        temp_list.append(point)


bottom_left_corner = find_sides('left', temp_list)
bottom_right_corner = find_sides('right', temp_list)

image = cv2.circle(image, bottom_left_corner, radius=5, thickness=-1, color=(255,0,0))
#image= cv2.putText(image, str(point), point, font, 0.6, (200, 0, 0))

image = cv2.circle(image, bottom_right_corner, radius=5, thickness=-1, color=(255,0,0))
#image = cv2.putText(image, str(point), point, font, 0.6, (200, 0, 0))

In [71]:
bottom_left_corner

[91, 676]

In [72]:
bottom_right_corner

[380, 636]

In [73]:
#check points at each level
left = []
right = []
for i in range(9):
    temp_list = []
    for point in global_approx:
        if (point[1] < max_height[1] - ((i+1)/10)*total_height) and (point[1] > max_height[1] - ((i+2)/10)*total_height):
            temp_list.append(point)

    left.append(find_sides('left', temp_list) if len(temp_list)>0 else None)
    right.append(find_sides('right', temp_list) if len(temp_list)>0 else None)


In [74]:
left

[None,
 [84, 500],
 [189, 469],
 [187, 374],
 [78, 344],
 None,
 [83, 204],
 None,
 [115, 59]]

In [75]:
right

[None,
 [395, 502],
 [190, 476],
 [300, 357],
 [392, 342],
 None,
 [358, 187],
 None,
 [360, 42]]

In [76]:
# middle threshold - average left to right from center between extremities
middle_threshold = 0.3

In [77]:
# radius = 0
# #convert distance to scale of 0-10
# pallet_center =int((bottom_right_corner[0] - bottom_left_corner[0])/2)+bottom_left_corner[0]

# for i, point in enumerate(left):
# 	above_list =[]
# 	below_list = []
# 	for entry in left[0:i]:
# 		if entry:
# 			below_list.append(entry)
# 	for entry in left[i:]:
# 		if entry:
# 			above_list.append(entry)

# 	if point:
# 		image = cv2.line(image, 
# 						(int(pallet_center - middle_threshold*pallet_center), 0), 
# 						(int(pallet_center - middle_threshold*pallet_center), 690), 
# 						(150,0,0), 1)
		
		
# 		if point[0] < pallet_center - middle_threshold*pallet_center:
# 			if point[0] < bottom_left_corner[0]:
				
# 				cv2.circle(image, point, radius=4, thickness=-1, color=(0,0,255))
# 				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))

# 			elif point[0] < find_sides('left', below_list)[0]:
# 				cv2.circle(image, point, radius=4, thickness=-1, color=(51,255,255))
# 				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))

# 			elif point[0] >= find_sides('left', above_list)[0]:
# 				cv2.circle(image, point, radius=4, thickness=-1, color=(0,255,0))
# 				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))


# for i, point in enumerate(right):
# 	above_list =[]
# 	below_list = []
# 	for entry in right[0:i]:
# 		if entry:
# 			below_list.append(entry)
# 	for entry in right[i:]:
# 		if entry:
# 			above_list.append(entry)

# 	if point:
# 		image = cv2.line(image, 
# 						(int(pallet_center + middle_threshold*pallet_center), 0), 
# 						(int(pallet_center + middle_threshold*pallet_center), 690), 
# 						(255,0,0), 1)


# 		if point[0] > pallet_center + middle_threshold*pallet_center:
# 			if point[0] > bottom_right_corner[0]:
# 				cv2.circle(image, point, radius=4, thickness=-1, color=(0,0,255))
# 				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))

# 			elif point[0] > find_sides('right', below_list)[0]:
# 				cv2.circle(image, point, radius=4, thickness=-1, color=(51,255,255))
# 				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))

# 			elif point[0] <= find_sides('right', above_list)[0]:
# 				cv2.circle(image, point, radius=4, thickness=-1, color=(0,255,0))
# 				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))




# # cv2.imshow('Contours', image)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()

In [78]:
radius = 0
#convert distance to scale of 0-10
pallet_center =int((bottom_right_corner[0] - bottom_left_corner[0])/2)+bottom_left_corner[0]

for i, point in enumerate(left):
	above_list =[]
	below_list = []
	for entry in left[0:i]:
		if entry:
			below_list.append(entry)
	for entry in left[i:]:
		if entry:
			above_list.append(entry)

	if point:
		image = cv2.line(image, 
						(int(pallet_center - middle_threshold*pallet_center), 0), 
						(int(pallet_center - middle_threshold*pallet_center), 690), 
						(150,0,0), 1)
		
		
		if point[0] < pallet_center - middle_threshold*pallet_center:
			if point[0] < bottom_left_corner[0]:
				if abs(point[0]-bottom_left_corner[0])<=5:
					cv2.circle(image, point, radius=7, thickness=-1, color=(0,255,0))
				else:
					cv2.circle(image, point, radius=7, thickness=-1, color=(0,0,255))
				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))

			elif point[0] < find_sides('left', below_list)[0]:
				cv2.circle(image, point, radius=7, thickness=-1, color=(51,255,255))
				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))

			elif point[0] >= find_sides('left', above_list)[0]:
				cv2.circle(image, point, radius=7, thickness=-1, color=(0,255,0))
				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))


for i, point in enumerate(right):
	above_list =[]
	below_list = []
	for entry in right[0:i]:
		if entry:
			below_list.append(entry)
	for entry in right[i:]:
		if entry:
			above_list.append(entry)

	if point:
		image = cv2.line(image, 
						(int(pallet_center + middle_threshold*pallet_center), 0), 
						(int(pallet_center + middle_threshold*pallet_center), 690), 
						(255,0,0), 1)


		if point[0] > pallet_center + middle_threshold*pallet_center:
			if point[0] > bottom_right_corner[0]:
				if abs(point[0]-bottom_right_corner[0])<=10:
					cv2.circle(image, point, radius=7, thickness=-1, color=(0,128,255))
				else:
					cv2.circle(image, point, radius=7, thickness=-1, color=(0,0,255))
				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))

			elif point[0] > find_sides('right', below_list)[0]:
				cv2.circle(image, point, radius=7, thickness=-1, color=(51,255,255))
				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))

			elif point[0] <= find_sides('right', above_list)[0]:
				cv2.circle(image, point, radius=7, thickness=-1, color=(0,255,0))
				# cv2.putText(image, str(point), point, font, 0.8, (200, 0, 0))


In [79]:
(bottom_right_corner[0] - bottom_left_corner[0])/2

144.5

In [80]:
filtered_left=list(filter(None,left))
left_top=find_sides("up",filtered_left)
left_top

[115, 59]

In [81]:
import math
def gradient(pt1,pt2):
    return (pt2[1]-pt1[1])/(pt2[0]-pt1[0])

def getangle(pointslist):
    pt1,pt2,pt3=pointslist[-3:]
    m1=gradient(pt1,pt2)
    m2=gradient(pt1,pt3)
    angR = math.atan((m2-m1)/(1+(m2*m1)))
    angD = round(math.degrees(angR))
    return angD
    

In [82]:
ang_points=[bottom_left_corner,left_top,[600,bottom_left_corner[1]]]
angD=getangle(ang_points)
print(angD)
diff=90-abs(angD)
op_str=f"The Boxes are off by {diff} degrees"
print(op_str)

88
Difference from 90deg is 2


In [83]:
image=cv2.line(image,bottom_left_corner,left_top,(255,255,51),1)
image=cv2.line(image,bottom_left_corner,(600,bottom_left_corner[1]),(255,255,51),1)
image=cv2.putText(image,str(angD),(bottom_left_corner[0]-40,bottom_left_corner[1]-20),cv2.FONT_HERSHEY_COMPLEX,1.5,(0,0,255),2)
image=cv2.putText(image,op_str,(0,20),cv2.FONT_HERSHEY_COMPLEX,0.4,(0,0,255),1)
cv2.imshow('image',image)
cv2.waitKey()
cv2.destroyAllWindows()

In [84]:
op_name='img'+img_path[-5:-4]

In [85]:

cv2.imwrite(f"D:\Lotus Labs Work\Anamoly Detection POC\Pallet Alignment\Output\{op_name}.jpg",image)

True